# 使用 fastNLP 进行序列标注

&emsp;&emsp;本篇教程将为您详细展示如何使用 fastNLP 使用 `WeiboNER` 数据集进行序列标注（Sequence labeling）任务。您可以使用 fastNLP 的各个组件快捷/方便地完成序列标注任务，达到出色的效果。 在阅读这篇教程前，希望您已经熟悉了fastNLP 的基础使用，尤其是数据的载入以及模型的构建。通过这个小任务，能让您进一步熟悉 fastNLP 的使用。

&emsp;&emsp;**本教程推荐使用 GPU 进行实验**

### 1. 命名实体识别(name entity recognition, NER)
&emsp;&emsp;命名实体识别任务是从文本中抽取出具有特殊意义或者指代性非常强的实体，通常包括人名、地名、机构名和时间等。 如下面的例子中：

```
我来自复旦大学。
```

&emsp;&emsp;其中 `复旦大学` 就是一个机构名，命名实体识别就是要从中识别出 `复旦大学` 这四个字是一个整体，且属于机构名这个类别。这个问题在实际做的时候会被转换为序列标注问题。

&emsp;&emsp;针对 `我来自复旦大学` 这句话，我们的预测目标将是 `[O, O, O, B-ORG, I-ORG, I-ORG, I-ORG]` ，其中 `O` 表示`out` ，即不是一个实体，`B-ORG` 是 ORG( organization的缩写) 这个类别的开头 (Begin) ，`I-ORG` 是 ORG 类别的中间(Inside)。

&emsp;&emsp;在本教程中我们将通过 fastNLP 尝试写出一个能够执行以上任务的模型。

### 2. 载入数据

&emsp;&emsp;在本篇教程中，我们选用微博实体命名任务。`WeiboNER` 数据集是基于微博的中文实体命名数据集，属于 [conll格式](https://universaldependencies.org/format.html) 的数据集。我们可以在[网络上](https://github.com/hltcoe/golden-horse/tree/master/data)找到该数据集的原始形式，其训练集的格式如下所示：

```txt
科	O
技	O
全	O
方	O
位	O
资	O
讯	O
智	O
能	O
，	O
快	O
捷	O
的	O
汽	O
车	O
生	O
活	O
需	O
要	O
有	O
三	O
屏	O
一	O
云	O
爱	O
你	O

对	O
，	O
输	O
给	O
一	O
个	O
女	B-PER.NOM
人	I-PER.NOM
，	O
的	O
成	O
绩	O
。	O
失	O
望	O

今	O
...
```

&emsp;&emsp;其实很简单，第一列代表文字，第二列代表这个词的标签，然后用一个空行来分割不同的句子，这就是 `conll` 格式的数据集。接下来我们会为您介绍不同的加载数据集的方式。

#### 2.1 使用内置的 Loader 加载数据
&emsp;&emsp;首先在 fastNLP 中，我们内置了加载该数据集的 `WeiboNERLoader` 来完成下载和格式转换。

In [1]:
from fastNLP.io import WeiboNERLoader
data_bundle = WeiboNERLoader().load()
print(data_bundle)
print(data_bundle.get_dataset('train')[:4])

/remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In total 3 datasets:
	dev has 270 instances.
	test has 270 instances.
	train has 1350 instances.

+------------------------------------------+------------------------------------------+
| raw_chars                                | target                                   |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+------------------------------------------+


&emsp;&emsp;假如您要使用的数据集也是 `conll` 格式的，那么也可以调用 fastNLP 的 `ConllLoader` 来直接加载，效果是一样的。不过缺点是所有 `Loader` 的 `load` 函数在只传入一个路径的情况下要求训练集、验证集和测试集的文件名分别包含 `train`、`dev` 和 `test`，否则 fastNLP 将会跳过对应的数据，您需要检查您的文件命名格式是否符合要求。比如我们可以从[这里](https://github.com/hltcoe/golden-horse/tree/master/data)下载 `WeiboNER` 数据集的三个原始文件 `weiboNERconll.train`、`weiboNER_conll.dev` 和 `weiboNER_conll.test` 并放在 `weibo/` 文件夹下，然后使用下面的方式加载数据：

In [2]:
from fastNLP.io import ConllLoader

data_bundle = ConllLoader(headers=["raw_words", "target"]).load("weibo/")
print(data_bundle)
print(data_bundle.get_dataset("train")[:4])

In total 3 datasets:
	dev has 270 instances.
	test has 270 instances.
	train has 1350 instances.

+------------------------------------------+------------------------------------------+
| raw_words                                | target                                   |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+------------------------------------------+


&emsp;&emsp;除此之外，`load` 函数还能够接受字典形式的输入，此时 `Loader` 对文件名的要求就没有那么严苛了，比如您想要用 `valid` 来命名验证集的话，就可以按如下方式加载：

In [3]:
from fastNLP.io import ConllLoader

data_bundle = ConllLoader(headers=["raw_words", "target"]).load({
    'train': 'weibo/weiboNER.conll.train',
    'test': 'weibo/weiboNER.conll.test',
    'valid': 'weibo/weiboNER.conll.dev'
})
print(data_bundle)
print(data_bundle.get_dataset("train")[:4])

In total 3 datasets:
	train has 1350 instances.
	test has 270 instances.
	valid has 270 instances.

+------------------------------------------+------------------------------------------+
| raw_words                                | target                                   |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+------------------------------------------+


#### 2.2 通过自定义 Loader 来加载数据

&emsp;&emsp;如果您想自己处理数据，您也可以通过继承 `Loader` 来自定义加载的方式。对于 `conll` 格式的数据，我们处理数据的目标就是：

1. 打开文件并顺序读入每一行，以空格分割，第一个是文字，第二个是标签
2. 遇到空行则记录一个新的句子

&emsp;&emsp;继续查看可以发现，三个文件中的数据格式都是一样的，那么我们可以继承一个 `Loader` 并重写 `_load` 方法来加载。不用担心，`Loader` 会遍历我们传入的几个路径将这些数据全都处理好的。

In [4]:
from fastNLP import DataSet, Instance
from fastNLP.io import Loader

# 继承Loader之后，我们只需要实现其中_load()方法，_load()方法传入一个文件路径，返回一个fastNLP DataSet对象，其目的是读取一个文件。
class MyWeiboNERLoader(Loader):
    def _load(self, path):
        ds = DataSet()
        # 打开文件
        with open(path, 'r') as f:
            segments = []
            for line in f:
                # 遍历每一行
                line = line.strip()
                if line == '':  # 如果为空行，说明需要切换到下一句了。
                    if segments:
                        # 如果句子部位空，则放入 raw_words 和 raw_target 中
                        raw_words = [s[0] for s in segments]
                        raw_target = [s[1] for s in segments]
                        # 将一个 sample 插入到 DataSet中
                        ds.append(Instance(raw_words=raw_words, raw_target=raw_target))  
                    segments = []
                else:
                    # 不是空行，则按空格分割放入 segments 中
                    parts = line.split()
                    assert len(parts) == 2
                    segments.append([parts[0], parts[-1]])
        return ds

# load 函数的参数也可以为一个字典，
data_bundle = MyWeiboNERLoader().load({
    'train': 'weibo/weiboNER.conll.train',
    'test': 'weibo/weiboNER.conll.test',
    'valid': 'weibo/weiboNER.conll.dev'
})
print(data_bundle)
print(data_bundle.get_dataset("train")[:4])

In total 3 datasets:
	train has 1350 instances.
	test has 270 instances.
	valid has 270 instances.

+------------------------------------------+------------------------------------------+
| raw_words                                | raw_target                               |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+------------------------------------------+


#### 2.3 其它的数据加载方式

&emsp;&emsp;数据处理的方法不是唯一的，只要能够得到一个 `DataBundle`，那就是可用的处理方法。您不必纠结于一定要用 `Loader` 来进行处理，它们只是帮助集成了一些常见的功能，如果您不习惯这样的方式或是对数据集格式有特殊要求，强行继承 `Loader` 反而会增加工作量和学习成本。

In [5]:
# 通过函数处理数据
from fastNLP.io import DataBundle

def load_data(path):
    ds = DataSet()
    # 打开文件
    with open(path, 'r') as f:
        segments = []
        for line in f:
            # 遍历每一行
            line = line.strip()
            if line == '':  # 如果为空行，说明需要切换到下一句了。
                if segments:
                    # 如果句子部位空，则放入 raw_words 和 raw_target 中
                    raw_words = [s[0] for s in segments]
                    raw_target = [s[1] for s in segments]
                    # 将一个 sample 插入到 DataSet中
                    ds.append(Instance(raw_words=raw_words, raw_target=raw_target))  
                segments = []
            else:
                # 不是空行，则按空格分割放入 segments 中
                parts = line.split()
                assert len(parts) == 2
                segments.append([parts[0], parts[-1]])
    return ds

data_bundle = DataBundle(datasets={
    "train": load_data("weibo/weiboNER.conll.train"),
    "dev": load_data("weibo/weiboNER.conll.dev"),
    "test": load_data("weibo/weiboNER.conll.test"),
})
print(data_bundle)
print(data_bundle.get_dataset("train")[:4])

In total 3 datasets:
	train has 1350 instances.
	dev has 270 instances.
	test has 270 instances.

+------------------------------------------+------------------------------------------+
| raw_words                                | raw_target                               |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+------------------------------------------+


### 3. 处理数据

&emsp;&emsp;接下来，我们需要对数据进行 **分词**。`fastNLP.transformers.torch` 下提供了一些常见的 Transformer 模型和 Tokenizer 来加载预训练的模型和分词器。

In [6]:
from fastNLP.transformers.torch import BertTokenizer
from fastNLP import cache_results, Vocabulary

@cache_results("weibo/weiboNER")
def process_data(data_bundle, model_name):

    tokenizer = BertTokenizer.from_pretrained(model_name)
    def bpe(raw_words):
        bpes = [tokenizer.cls_token_id]
        first = [0]
        first_index = 1  # 记录第一个bpe的位置
        for word in raw_words:
            bpe = tokenizer.encode(word, add_special_tokens=False)
            bpes.extend(bpe)
            first.append(first_index)
            first_index += len(bpe)
        bpes.append(tokenizer.sep_token_id)
        first.append(first_index)
        return {'input_ids': bpes, 'input_len': len(bpes), 'first': first, 'seq_len': len(raw_words)}
    # 对data_bundle中每个dataset的每一条数据中的raw_words使用bpe函数，并且将返回的结果加入到每条数据中。
    data_bundle.apply_field_more(bpe, field_name='raw_words', num_proc=4)

    # tag的词表，由于这是词表，所以不需要有padding和unk
    tag_vocab = Vocabulary(padding=None, unknown=None)
    # 从 train 数据的 raw_target 中获取建立词表
    tag_vocab.from_dataset(data_bundle.get_dataset('train'), field_name='raw_target')
    # 使用词表将每个 dataset 中的raw_target转为数字，并且将写入到target这个field中
    tag_vocab.index_dataset(data_bundle.datasets.values(), field_name='raw_target', new_field_name='target')

    # 可以将 vocabulary 绑定到 data_bundle 上，方便之后使用。
    data_bundle.set_vocab(tag_vocab, field_name='target')

    return data_bundle, tokenizer

data_bundle, tokenizer = process_data(data_bundle, 'bert-base-chinese')
print(data_bundle)
print(data_bundle.get_dataset("train")[:4])

[08:55:38] INFO     Read cache from /remote-home/shxing/fnlp-doc/fastNLP ]8;id=598668;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/utils/cache_results.py\cache_results.py]8;;\:]8;id=51913;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/utils/cache_results.py#320\320]8;;\
                    /docs/source/tutorials/torch/weibo/33ac7cae_weiboNER                     
                     (Saved on 2022-07-24 22:59:52.436694).                                  

In total 3 datasets:
	train has 1350 instances.
	dev has 270 instances.
	test has 270 instances.
In total 1 vocabs:
	target has 15 entries.

+----------------+----------------+----------------+-----------+----------------+---------+----------------+
| raw_words      | raw_target     | input_ids      | input_len | first          | seq_len | target         |
+----------------+----------------+----------------+-----------+----------------+---------+----------------+
| ['科', '技'... | ['O', 'O', ... | [101, 4906,... | 28        | [0, 1, 2, 3... | 26      | [0, 0, 0, 0... |
| ['对', '，'... | ['O', 'O', ... | [101, 2190,... | 17        | [0, 1, 2, 3... | 15      | [0, 0, 0, 0... |
| ['今', '天'... | ['O', 'O', ... | [101, 791, ... | 81        | [0, 1, 2, 3... | 79      | [0, 0, 0, 0... |
| ['今', '年'... | ['O', 'O', ... | [101, 791, ... | 20        | [0, 1, 2, 3... | 18      | [0, 0, 0, 0... |
+----------------+----------------+----------------+-----------+----------------+---------+-------------

&emsp;&emsp;最后将数据放入 DataLoder 即可。

In [7]:
from fastNLP import prepare_torch_dataloader

dataloaders = prepare_torch_dataloader(data_bundle, batch_size=12)

for dl in dataloaders.values():
    # 可以通过 set_pad 修改 padding 的行为。
    dl.set_pad('input_ids', pad_val=tokenizer.pad_token_id)
    dl.set_pad('target', pad_val=-100)

&emsp;&emsp;`prepare_dataloader` 函数会为 `data_bundle` 中的所有数据集分配一个 DataLoader。这是一种简便的方法，但是如果您想对三种数据集使用不同的 batch_size 和 shuffle 等设置，也可以分开构建，如下所示：

In [8]:
from torch.utils.data import DataLoader

# 直接使用 torch 的 DataLoader
train_dataloader = DataLoader(data_bundle.get_dataset("train"), batch_size=12, shuffle=True)
dev_dataloader = DataLoader(data_bundle.get_dataset("dev"), batch_size=12, shuffle=False)
test_dataloader = DataLoader(data_bundle.get_dataset("test"), batch_size=1, shuffle=False)

from fastNLP import prepare_torch_dataloader

# 调用 prepare_torch_dataloader
train_dataloader = prepare_torch_dataloader(data_bundle.get_dataset("train"), batch_size=12, shuffle=True)
dev_dataloader = prepare_torch_dataloader(data_bundle.get_dataset("dev"), batch_size=12, shuffle=False)
test_dataloader = prepare_torch_dataloader(data_bundle.get_dataset("test"), batch_size=1, shuffle=False)

### 4. 模型准备

&emsp;&emsp;我们选用最基础的 Bert 模型来进行训练，并使用预训练模型 `bert-base-chinese` 。模型的 `forward` 函数将输入首先经过 Bert 模型，接着经过两个线性层得到每个字在每种标签上的权重。

In [9]:
import torch
from torch import nn
from fastNLP.transformers.torch import BertModel
from fastNLP import seq_len_to_mask
import torch.nn.functional as F


class BertNER(nn.Module):
    def __init__(self, model_name, num_class):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.mlp = nn.Sequential(nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size),
                                nn.Dropout(0.3),
                                nn.Linear(self.bert.config.hidden_size, num_class))
    
    def forward(self, input_ids, input_len, first):
        attention_mask = seq_len_to_mask(input_len)
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        first = first.unsqueeze(-1).repeat(1, 1, last_hidden_state.size(-1))
        first_bpe_state = last_hidden_state.gather(dim=1, index=first)
        first_bpe_state = first_bpe_state[:, 1:-1]  # 删除 cls 和 sep
        
        pred = self.mlp(first_bpe_state)
        return {'pred': pred}
    
    def train_step(self, input_ids, input_len, first, target):
        pred = self(input_ids, input_len, first)['pred']
        loss = F.cross_entropy(pred.transpose(1, 2), target)
        return {'loss': loss}
    
    def evaluate_step(self, input_ids, input_len, first):
        pred = self(input_ids, input_len, first)['pred'].argmax(dim=-1)
        return {'pred': pred}

model = BertNER('bert-base-chinese', len(data_bundle.get_vocab('target')))

[08:55:42] WARNING  Some weights of the model checkpoint at            ]8;id=879502;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=496206;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1490\1490]8;;\
                    bert-base-chinese were not used when initializing                        
                    BertModel: ['cls.predictions.bias',                                      
                    'cls.predictions.transform.LayerNorm.bias',                              
                    'cls.predictions.transform.LayerNorm.weight',                            
                    'cls.seq_relationship.weight',                                           
                    'cls.predictions.transform.dense.weight',                                
                    'cls.predictions.transform.dense.bias',                                  
                    'cls.seq_relationship.bias',                                             
                    'cls.predictions.decoder.weight']                                        
                    - This IS expected if you are initializing                               
                    BertModel from the checkpoint of a model trained                         
                    on another task or with another architecture (e.g.                       
                    initializing a BertForSequenceClassification model                       
                    from a BertForPreTraining model).                                        
                    - This IS NOT expected if you are initializing                           
                    BertModel from the checkpoint of a model that you                        
                    expect to be exactly identical (initializing a                           
                    BertForSequenceClassification model from a                               
                    BertForSequenceClassification model).                                    

           INFO     All the weights of BertModel were initialized from ]8;id=897263;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=62912;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1507\1507]8;;\
                    the model checkpoint at bert-base-chinese.                               
                    If your task is similar to the task the model of                         
                    the checkpoint was trained on, you can already use                       
                    BertModel for predictions without further                                
                    training.                                                                

### 5. 开始训练

#### 5.1 使用 Trainer 进行训练

&emsp;&emsp;在 Callback 上我们选用 `LoadBestModelCallback` 和 `TorchWarmupCallback` 。前者可以在训练中找到结果最好的权重并在结束后加载回来，其监控的结果值可以在初始化 `LoadBestModelCallback` 时指定，也可以在 `Trainer` 中设置，即 `monitor='f#f'` 参数；后者能够对学习率进行预热，对于 Bert 这样的复杂模型很有效。

In [10]:
from torch import optim
from fastNLP import Trainer, LoadBestModelCallback, TorchWarmupCallback
from fastNLP import SpanFPreRecMetric

optimizer = optim.Adam(model.parameters(), lr=2e-5)
callbacks = [
    LoadBestModelCallback(),
    TorchWarmupCallback(),
]
metrics = {
    "f": SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target')),
}

trainer = Trainer(model=model, train_dataloader=dataloaders['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dataloaders['dev'], 
                  metrics=metrics, n_epochs=10, callbacks=callbacks, 
                  device=0, monitor='f#f')
trainer.run()

[08:55:49] INFO     Running evaluator sanity check for 2 batches.              ]8;id=135149;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=278048;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

/remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/rich/live.py:221: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

+++++++++++++++++++++++++++++ Eval. results on Epoch:1, Batch:0 +++++++++++++++++++++++++++++

{
  "f#f": 0.0,
  "pre#f": 0.0,
  "rec#f": 0.0
}

+++++++++++++++++++++++++++++ Eval. results on Epoch:2, Batch:0 +++++++++++++++++++++++++++++

...

{
  "f#f": 0.513471,
  "pre#f": 0.496933,
  "rec#f": 0.531148
}

---------------------------- Eval. results on Epoch:10, Batch:0 -----------------------------

{
  "f#f": 0.511041,
  "pre#f": 0.492401,
  "rec#f": 0.531148
}

[08:57:39] INFO     The best performance for monitor f#f:0.527546 was ]8;id=432902;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=453694;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    achieved in Epoch:8, Global Batch:904. The                               
                    evaluation result:                                                       
                    {'f#f': 0.527546, 'pre#f': 0.537415, 'rec#f':                            
                    0.518033}                                                                

           INFO     Loading best model from buffer with f#f:  ]8;id=513700;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=947866;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#115\115]8;;\
                    0.527546...                                                              

#### 5.2 输出标注的结果

&emsp;&emsp;在训练完成后，我们可以使用 `Evaluator` 来输出标注的结果，在这里我们选择使用 `evaluate_batch_step_fn` 参数来实现这一功能，`evaluate_batch_step_fn` 函数必须接受两个参数 `evaluator` 和 `batch` 。在 `output_labeling` 函数中，我们将 `evaluate_step` 后的结果在词表中进行转换得到字符串形式的标签，并且将原标签和预测的标签一起输出来展示模型的效果。由于我们之前为 dataloader 设置的 batch_size 是 12，因此这里仅运行一个 batch 来展示，防止输出过长。在 `output_labeling` 函数中，您可以进行更加灵活的设置，比如将结果保存起来。

In [11]:
from fastNLP import Evaluator, print

def output_labeling(evaluator, batch):
    outputs = evaluator.evaluate_step(batch)["pred"]
    raw_words, raw_targets = batch["raw_words"], batch["raw_target"]
    for words, raw_target, output in zip(raw_words, raw_targets, outputs):

        print("sentence:", words)
        labels = [data_bundle.get_vocab("target").idx2word[idx] for idx in output[:len(words)].tolist() ]
        print("labels:", labels)
        print("target:", raw_target)

evaluator = Evaluator(model=model, dataloaders=dataloaders["test"], 
                      device=0, evaluate_batch_step_fn=output_labeling)
evaluator.run(1)

Output()

[08:58:06] INFO     sentence: ['一', '节', '课', '的', '时', '间', '真', '心', ]8;id=735907;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=640687;file:///tmp/ipykernel_2969/236964521.py#8\8]8;;\
                    '感', '动', '了', '李', '开', '复', '感', '动']                          

           INFO     labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',     ]8;id=763056;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=724218;file:///tmp/ipykernel_2969/236964521.py#10\10]8;;\
                    'O', 'O', 'B-PER.NAM', 'I-PER.NAM', 'I-PER.NAM', 'O',                    
                    'O']                                                                     

           INFO     target: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',     ]8;id=557590;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=884499;file:///tmp/ipykernel_2969/236964521.py#11\11]8;;\
                    'O', 'O', 'B-PER.NAM', 'I-PER.NAM', 'I-PER.NAM', 'O',                    
                    'O']                                                                     

           INFO     sentence: ['回', '复', '支', '持', '，', '赞', '成', '，', ]8;id=25597;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=681057;file:///tmp/ipykernel_2969/236964521.py#8\8]8;;\
                    '哈', '哈', '米', '八', '吴', '够', '历', '史', '要',                    
                    '的', '陈', '小', '奥', '丁', '丁', '我', '爱', '小',                    
                    '肥', '肥', '一', '族', '大', '头', '仔', '大', '家',                    
                    '团', '结', '一', '致', '，', '誓', '要', '去', '台',                    
                    '湾', '饮', '喜', '酒', '，', '由', '包', '机', '，',                    
                    '团', '结', '的', '力', '量', '大']                                      

...

           INFO     target: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',     ]8;id=717776;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=190442;file:///tmp/ipykernel_2969/236964521.py#11\11]8;;\
                    'O', 'O', 'O', 'O']                                                      

           INFO     sentence: ['叶', '赟', '葆', '：', '全', '球', '时', '尚', ]8;id=119877;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=455054;file:///tmp/ipykernel_2969/236964521.py#8\8]8;;\
                    '财', '运', '滚', '滚', '而', '来', '钱']                                

           INFO     labels: ['B-PER.NAM', 'I-PER.NAM', 'I-PER.NAM', 'O', 'O', ]8;id=684954;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=857535;file:///tmp/ipykernel_2969/236964521.py#10\10]8;;\
                    'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']                        

           INFO     target: ['B-PER.NAM', 'I-PER.NAM', 'I-PER.NAM', 'O', 'O', ]8;id=44835;file:///tmp/ipykernel_2969/236964521.py\236964521.py]8;;\:]8;id=58403;file:///tmp/ipykernel_2969/236964521.py#11\11]8;;\
                    'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']                        

{}

&emsp;&emsp;除了使用 `evaluate_batch_step_fn` 参数之外，您也可以使用 `evaluate_fn` 参数来处理单个 batch 的数据。该参数您可以查阅 [Evaluator的文档](../../api/generated/fastNLP.core.Evaluator.rst) 来了解。简言之，在使用该参数的时候您需要在模型中定义一个函数，然后将该函数的名字作为 `evaluate_fn` 参数的值，不设置的情况下相当于 `evaluate_fn='evaluate_step'` 。

&emsp;&emsp;如果您想要批量地处理输出或进行保存，也可以使用 **Metric** 。自定义 Metric `OutputSaver` 在每个 batch 得到结果后会执行 `update` 函数，将原文、原标签和预测的标签放入成员中，然后在 `get_metric` 函数中将其一股脑写入文件中。`update` 函数的三个参数中，pred 参数来自 `evaluate_step` 函数的返回值，而剩下两个则源于我们的输入。如您所见 **Metric** 也具有很高的灵活性，与 Callback 配合起来它们几乎可以满足所有常见的定制需求。

In [12]:
from fastNLP import Metric

class OutputSaver(Metric):
    def __init__(self, tag_vocab, filename):
        super(OutputSaver, self).__init__()
        self.tag_vocab = tag_vocab
        self.filename = filename
        self.words_list = []
        self.targets_list = []
        self.labels_list = []

    def reset(self):
        self.words_list = []
        self.targets_list = []
        self.labels_list = []

    def update(self, pred, raw_words, raw_target):
        for words, single_raw_target, output in zip(raw_words, raw_target, pred):
            labels = [data_bundle.get_vocab("target").idx2word[idx] for idx in output[:len(words)].tolist() ]
            self.words_list.append(words)
            self.targets_list.append(single_raw_target)
            self.labels_list.append(labels)

    def get_metric(self):
        with open(self.filename, "w") as f:
            # 逐行写入文件
            for words, targets, labels in zip(self.words_list, self.targets_list, self.labels_list):
                f.write(" ".join(words) + "\n")
                f.write(" ".join(labels) + "\n")
                f.write(" ".join(targets) + "\n")
                f.write("\n")


evaluator = Evaluator(model=model, dataloaders=dataloaders["test"], 
                      device=0, n_epochs=1,
                      metrics={"output": OutputSaver(data_bundle.get_vocab("target"), "output.txt")})
evaluator.run()

Output()

[08:58:08] WARNING  Metric:output returns None when getting metric results.  ]8;id=519634;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/evaluator.py\evaluator.py]8;;\:]8;id=118548;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/evaluator.py#529\529]8;;\

{}